In [1]:
ll ../../

total 20
drwxrwxr-x 4 jharsh 4096 Feb 18 20:06 comet/
drwxrwxr-x 3 jharsh 4096 Feb 11 06:38 data/
drwxrwxr-x 3 jharsh 4096 Feb 17 17:53 models/
-rw-rw-r-- 1 jharsh 1979 Feb 11 12:43 README.md
drwxrwxr-x 2 jharsh 4096 Feb  2 21:11 utils/


In [2]:
import os
import sys
import argparse
import torch

sys.path.append(os.getcwd())

# import src.data.data as data
import src.data.config as cfg
import src.interactive.functions as interactive



In [3]:
class Args:
    def __init__(self, 
                 device = 'cpu',
                 model_file="pretrained_models/atomic_pretrained_model.pickle",
                 sampling_algorithm = 'beam-5'
                ):
        self.device = device
        self.model_file = model_file
        self.sampling_algorithm = sampling_algorithm

In [4]:
args = Args()

In [5]:

opt, state_dict = interactive.load_model_file(args.model_file)

data_loader, text_encoder = interactive.load_data("atomic", opt)

n_ctx = data_loader.max_event + data_loader.max_effect
n_vocab = len(text_encoder.encoder) + n_ctx
model = interactive.make_model(opt, n_vocab, n_ctx, state_dict)

if args.device != "cpu":
    cfg.device = int(args.device)
    cfg.do_gpu = True
    torch.cuda.set_device(cfg.device)
    model.cuda(cfg.device)
else:
    cfg.device = "cpu"


Loading data from: data/atomic/processed/generation/categories_oEffect#oReact#oWant#xAttr#xEffect#xIntent#xNeed#xReact#xWant-maxe1_17-maxe2_35-maxr_1.pickle
52


In [6]:
sampling_algorithm = args.sampling_algorithm

def fnc(persona, debug=False): # list of sentences

    ret = []
    for sent in persona:
        
        cur_sent = {'sent':sent}
        if debug:
            print()
            print("-x"*33)
            print("=====>>>> sent = ", sent)

        input_event = sent
        category = "all"
        
        if debug:
            print()
            print("category = ", category)

        sampler = interactive.set_sampler(opt, sampling_algorithm, data_loader)
        outputs = interactive.get_atomic_sequence(
            input_event, model, sampler, data_loader, text_encoder, category)
        # print("outputs = ", outputs)
        cur_sent['comet']  = outputs
        ret.append(cur_sent)

    return ret


In [7]:
import json
import random

DATA_FILE = '../../data/personachat_self_original.json'

class AnnotatePersonaChat():
    
    def __init__(self):
        with open(DATA_FILE, "r", encoding="utf-8") as read_file:
            self.data = json.load(read_file)

        print('Read {} training examples and {} validation examples'.format(
            len(self.data['train']), len(self.data['valid'])
        ))

    def get_sample(self, split='train'):
        split = self.data[split]
        index = random.randint(0, len(split))
        sample = split[index]

        persona = sample['personality']
        utterances = sample['utterances']

        conversation = utterances[-1]['history'] + [utterances[-1]['candidates'][-1]]

        print('PERSONA {}\n{}'.format(
            '='*33, '\n'.join(persona))
        )
        print('CONVERSATION {}\n- {}'.format(
            '='*33, '\n- '.join(conversation))
        )
        return persona

        
    def process_all(self, dump_fname):
        self.annotated_data = {}
        for split,data in self.data.items():
            annotated_data = []
            for row in data:
                row['coment_annotation'] = fnc(row['personality'])
                annotated_data.append(row)
            self.annotated_data[split] = annotated_data
        json.dump(self.annotated_data, open(dump_fname,'w'))


In [8]:
solver = AnnotatePersonaChat()

Read 17878 training examples and 1000 validation examples


In [9]:
persona_sample = solver.get_sample('train')

PERSONA =================================
i want to move .
i don t like feeling controlled .
i don t like my reputation .
i wish i could take back a mistake .
i've a harsh inner critic .
CONVERSATION =================================
- __ SILENCE __
- hi how are you today ?
- good ! i just ate a bunch of green apples . my favorite . how are you ?
- what other fruits do you like ?
- pears , raisins . i like most fruits .
- that is good i love apples and grapes strawberries and watermelon too .
- i really don't like meat tho .
- really i love meat does your diet consist of veggies ?
- yes . i also eat a lot of fruit . apples . even with the skin on them .
- the skin is good has lots of the vitamins and minerals
- this is true . so , what do you do for work ?
- i work in a daycare center with the babies what do you do for work ?
- i manage a small it department .
- that is good is it stressful ?


In [10]:
persona_sample

['i want to move .',
 'i don t like feeling controlled .',
 'i don t like my reputation .',
 'i wish i could take back a mistake .',
 "i've a harsh inner critic ."]

In [11]:
fnc(persona_sample)

Input Event:   i want to move .
Target Effect: oEffect

Candidate Sequences:
none
they get evicted from their home
they get evicted from their house
they get evicted
gets evicted


Input Event:   i want to move .
Target Effect: oReact

Candidate Sequences:
none
happy
interested
excited
surprised


Input Event:   i want to move .
Target Effect: oWant

Candidate Sequences:
none
to move with personx
to sell the house
to help them move
to move with them


Input Event:   i want to move .
Target Effect: xAttr

Candidate Sequences:
adventurous
restless
motivated
decisive
brave


Input Event:   i want to move .
Target Effect: xEffect

Candidate Sequences:
none
thinks about moving
personx moves
gets evicted
moves


Input Event:   i want to move .
Target Effect: xIntent

Candidate Sequences:
to live somewhere else
to move
to be somewhere else
to get away
to get away from something


Input Event:   i want to move .
Target Effect: xNeed

Candidate Sequences:
none
to find a place to live
to find a 

Input Event:   i've a harsh inner critic .
Target Effect: oWant

Candidate Sequences:
none
to listen to personx
to argue with personx
to listen to person x
to listen to personx 's opinion


Input Event:   i've a harsh inner critic .
Target Effect: xAttr

Candidate Sequences:
opinionated
judgmental
mean
rude
insightful


Input Event:   i've a harsh inner critic .
Target Effect: xEffect

Candidate Sequences:
none
personx is criticized
is criticized
gets into trouble
personx gets into trouble


Input Event:   i've a harsh inner critic .
Target Effect: xIntent

Candidate Sequences:
none
to be heard
to make a point
to be understood
to be a jerk


Input Event:   i've a harsh inner critic .
Target Effect: xNeed

Candidate Sequences:
none
to have a bad experience
to think about something
to learn about people
to think about something else


Input Event:   i've a harsh inner critic .
Target Effect: xReact

Candidate Sequences:
angry
like he is being rude
frustrated
satisfied
bad


Input Event: 

[{'sent': 'i want to move .',
  'comet': {'oEffect': {'event': 'i want to move .',
    'effect_type': 'oEffect',
    'beams': ['none',
     'they get evicted from their home',
     'they get evicted from their house',
     'they get evicted',
     'gets evicted'],
    'beam_losses': [-0.03018808364868164,
     -1.7714513540267944,
     -1.8861513137817383,
     -1.9851768016815186,
     -2.0106801986694336]},
   'oReact': {'event': 'i want to move .',
    'effect_type': 'oReact',
    'beams': ['none', 'happy', 'interested', 'excited', 'surprised'],
    'beam_losses': [-0.051430702209472656,
     -2.2178080081939697,
     -2.7862606048583984,
     -2.8557322025299072,
     -2.88124942779541]},
   'oWant': {'event': 'i want to move .',
    'effect_type': 'oWant',
    'beams': ['none',
     'to move with personx',
     'to sell the house',
     'to help them move',
     'to move with them'],
    'beam_losses': [-0.052761077880859375,
     -1.4152154922485352,
     -1.4859815835952759,
   